<a href="https://colab.research.google.com/github/pedroarthurob/NLP-FinalProject/blob/main/Projeto_Final_PLN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install 'crewai[tools]'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 1.4 MB/s eta 0:00:00
  Attempting uninstall: crewai-tools
    Found existing installation: crewai-tools 0.1.6
    Uninstalling crewai-tools-0.1.6:
      Successfully uninstalled crewai-tools-0.1.6


In [8]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [10]:
import os
from google.colab import userdata

In [15]:
OPENAI_API_KEY = userdata.get('OPEN_AI_KEY')
model_ID = userdata.get('GPT_MODEL')


os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["OPENAI_MODEL_NAME"] = model_ID

In [ ]:
from crewai_tools import PDFSearchTool
from crewai import Agent, Crew, Process, Task

In [ ]:
pdf_tool = PDFSearchTool(directory='/content/enem_provas_gabaritos/provas')

In [ ]:
# Definindo o agente que pesquisa tópicos
topic_researcher = Agent(
    role='Topic Researcher',
    goal='Search for relevant information on the topic {topic} from the provided ENEM PDF files.',
    verbose=True,
    memory=True,
    backstory='Expert in analyzing exam content and identifying key topics based on previous ENEM exams.',
    tools=[pdf_tool],
    allow_delegation=True
)

In [ ]:
# Definindo o agente que cria o plano de estudos
study_plan_creator = Agent(
    role='Study Plan Creator',
    goal='Create a personalized study plan for the user based on their selected topics.',
    verbose=True,
    memory=True,
    backstory='Expert in educational planning and optimizing study sessions for exams.',
)

In [ ]:
# Definindo o agente que sugere conjuntos de assuntos diários
daily_subject_suggester = Agent(
    role='Daily Subject Suggester',
    goal='Suggest daily subjects and questions for the user based on their study plan.',
    verbose=True,
    memory=True,
    backstory='Expert in daily topic organization and question generation for effective studying.'
)

In [ ]:
# Definindo a tarefa de pesquisa
research_task = Task(
    description="Identify and analyze relevant topics from the provided ENEM PDF files related to {topic}.",
    expected_output="A detailed report on the relevant topics and their occurrence in the exam.",
    agent=topic_researcher,
    tools=[pdf_tool]
)

In [ ]:
# Definindo a tarefa para criar o plano de estudos
study_plan_task = Task(
    description="Create a personalized study plan based on the topics selected by the user.",
    expected_output="A structured study plan with daily topics, questions, and resources.",
    agent=study_plan_creator
)

In [ ]:
# Definindo a tarefa para sugerir assuntos diários
daily_subject_task = Task(
    description="Suggest daily subjects and questions based on the user's study plan.",
    expected_output="A list of daily topics and corresponding questions to be studied.",
    agent=daily_subject_suggester
)

In [ ]:
# Criando a crew
my_crew = Crew(
    agents=[topic_researcher, study_plan_creator, daily_subject_suggester],
    tasks=[research_task, study_plan_task, daily_subject_task],
    verbose=True,
    process=Process.sequential,
    memory=True,
    cache=True,
    max_rpm=100,
    share_crew=True
)

In [ ]:
# Função para iniciar a pesquisa
def research_topic():
    topic = input("Por favor, informe o tópico que você gostaria de estudar: ")
    my_crew.run(task_name='research_task', topic=topic)
    print("Pesquisa concluída. Verifique os resultados para o tópico:", topic)

# Função para criar o plano de estudos
def create_study_plan():
    user_topics = input("Informe os tópicos que deseja incluir no plano de estudos (separados por vírgula): ").split(',')
    user_topics = [t.strip() for t in user_topics]  # Remover espaços em branco
    study_plan_creator.run(task_name='study_plan_task', topics=user_topics)
    print("Plano de estudos criado com sucesso!")

# Função para sugerir assuntos diários
def suggest_daily_subjects():
    daily_subject_suggester.run(task_name='daily_subject_task')
    print("Sugestões de assuntos diários geradas com sucesso!")

In [ ]:
# Menu interativo
def menu():
    while True:
        print("\nMenu:")
        print("1. Pesquisar tópico")
        print("2. Criar plano de estudos")
        print("3. Sugerir assuntos diários")
        print("4. Sair")

        choice = input("Escolha uma opção (1-4): ")

        if choice == '1':
            research_topic()
        elif choice == '2':
            create_study_plan()
        elif choice == '3':
            suggest_daily_subjects()
        elif choice == '4':
            print("Saindo do programa...")
            break
        else:
            print("Opção inválida! Tente novamente.")

# Executando o menu
menu()